--- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [1]:
example_schematic_str = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""

In [57]:
import re



def process_scheme_line(scheme_line_str: str) -> tuple[set[int], list[re.Match], set[int]]:
    """
    Returns a tuple of list of Matches for symbols and list of Matches for numbers
    """
    special_matches = set()
    gear_matches = set()
    for char_idx, char_val in enumerate(scheme_line_str):
        if char_val == "*":
            gear_matches.add(char_idx)
        if not (char_val.isnumeric() or char_val == "."):
            special_matches.add(char_idx)
    
    special_matches_plus_adjacent = set()
    for smi in special_matches:
        min_trunc = max(0, smi - 1)
        max_trunc = min(len(scheme_line_str) - 1, smi + 1)
        special_matches_plus_adjacent = special_matches_plus_adjacent.union({min_trunc, smi, max_trunc})
    digits_matches = list(re.finditer(r"(\d+)", scheme_line_str))

    return special_matches_plus_adjacent, digits_matches, gear_matches


def find_digits_adjacent_from_trio_lines(line_trio: list[tuple[set[int], list[re.Match]]]):
    indices_included = set()
    output_sum = 0
    gear_ratio_sum = 0
    all_digit_matches = []
    for special_indices, digit_matches, _ in line_trio:
        indices_included = indices_included.union(special_indices)
        all_digit_matches.extend(digit_matches)

    for digit_matches in line_trio[1][1]:
        if set(range(*digit_matches.span())).intersection(set(indices_included)):
            output_sum += int(digit_matches.group(1))

    for gear_index in line_trio[1][2]:
        adjacent_numbers = []
        adjacent_indices = set([max(0, gear_index - 1), gear_index, gear_index + 1])
        for anyline_digit_match in all_digit_matches:
            if set(range(*anyline_digit_match.span())).intersection(adjacent_indices):
                adjacent_numbers.append(int(anyline_digit_match.group(1)))
        if len(adjacent_numbers) == 2:
            gear_ratio_sum += adjacent_numbers[0] * adjacent_numbers[1]

    return output_sum, gear_ratio_sum



def process_engine_scheme_string(scheme_str: str) -> tuple[int, int]:
    scheme_lines_list = scheme_str.strip().split("\n")
    total_sum = 0
    gear_ratio_sum = 0
    process_line_trio = [
        process_scheme_line(""),
        process_scheme_line(scheme_lines_list[0]),
        process_scheme_line(scheme_lines_list[1]),
    ]
    added_part_sum, added_gear_sum = find_digits_adjacent_from_trio_lines(process_line_trio)
    total_sum += added_part_sum
    gear_ratio_sum += added_gear_sum
    for scheme_line in scheme_lines_list[2:]:
        process_line_trio.pop(0)
        process_line_trio.append(process_scheme_line(scheme_line))
        added_part_sum, added_gear_sum = find_digits_adjacent_from_trio_lines(process_line_trio)
        total_sum += added_part_sum
        gear_ratio_sum += added_gear_sum

    # process last line
    process_line_trio.pop(0)
    process_line_trio.append(process_scheme_line(""))
    added_part_sum, added_gear_sum = find_digits_adjacent_from_trio_lines(process_line_trio)
    total_sum += added_part_sum
    gear_ratio_sum += added_gear_sum

    return total_sum, gear_ratio_sum




In [58]:
assert process_engine_scheme_string(example_schematic_str)[0] == 4361

In [59]:
with open("../inputs/day3_input.txt") as f:
    input_str = f.read()


In [60]:
process_engine_scheme_string(input_str)

(512794, 67779080)

# Part 2

--- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [62]:
assert process_engine_scheme_string(example_schematic_str)[1] == 467835

In [63]:
process_engine_scheme_string(input_str)[1]

67779080